In [ ]:
import pyodbc
import getpass
import pandas as pd
from os import path

In [ ]:
dir_data = '/path/to/data/dir'
file_concepts = path.join(dir_data, 'concepts.csv')
file_persons = path.join(dir_data, 'persons.csv')

# SQL server config
sql_config = {
    'driver': '',
    'server': '',
    'database': '',
    'uid': ''
}

pwd=getpass.getpass()

In [ ]:
conn = pyodbc.connect(**sql_config, pwd=pwd)
cursor = conn.cursor()

# Get Metadata

Get concept definitions

In [ ]:
sql = """SELECT concept_id, concept_name, domain_id, vocabulary_id, concept_class_id, standard_concept, concept_code
    FROM concept;"""
df_concpets = pd.read_sql(sql, conn)
df_concpets.to_csv(file_concepts, sep='\t', na_rep='NULL', header=True, index=False)

# Get patient data

### Patient data extraction without data cleaning (default)

In [ ]:
sql = """SELECT person_id, gender_concept_id, CONVERT(DATE, birth_datetime) AS birth_date, race_concept_id, ethnicity_concept_id 
    FROM person"""
df_persons = pd.read_sql(sql, conn)
df_persons.to_csv(file_persons, sep='\t', na_rep='NULL', header=True, index=False)
n_persons = len(df_persons.index)

### Patient data extraction with data cleaning

In [ ]:
# Get the highest ancestors of race_concept_id because our database has many asians recorded by their nationality
# Also exclude patients whose birth_datetime is a known bad birth_datetime
sql = """SELECT person_id, gender_concept_id, CONVERT(DATE, p.birth_datetime) AS birth_date, r.ancestor_concept_id AS race_concept_id, ethnicity_concept_id 
    FROM person p
    JOIN user_schema.dbo.race_ancestor_concepts r ON p.race_concept_id = r.race_concept_id
    LEFT JOIN user_schema.dbo.ohdsi_2019q2_bad_dobs b ON p.birth_datetime = b.birth_datetime
    WHERE b.birth_datetime IS NULL AND p.year_of_birth >= 1900;"""

df_persons = pd.read_sql(sql, conn)
df_persons.to_csv(file_persons, sep='\t', na_rep='NULL', header=True, index=False)
n_persons = len(df_persons.index)

## Get sequence for each patient

In [ ]:
# Read df_persons back in from file
df_persons = pd.read_csv(file_persons, sep='\t', header=0)
n_persons = len(df_persons.index)

In [ ]:
import time
t1 = time.time()

sql = """SELECT x.start_date, STRING_AGG(x.concept_id, ',') AS concept_ids
FROM 
    ((SELECT co.condition_concept_id AS concept_id, co.condition_start_date AS start_date
    FROM dbo.condition_occurrence co
    LEFT JOIN user_schema.dbo.iatrogenic_codes_with_desc i ON co.condition_concept_id = i.concept_id
    JOIN concept c ON co.condition_concept_id = c.concept_id
    LEFT JOIN visit_occurrence v ON co.visit_occurrence_id = v.visit_occurrence_id
    WHERE co.person_id = ? AND co.condition_concept_id != 0 AND i.concept_id IS NULL AND co.condition_start_date >= ? 
        AND c.domain_id = 'Condition' AND (v.visit_occurrence_id IS NULL OR co.condition_start_date <= v.visit_end_date)
    GROUP BY co.condition_concept_id, co.condition_start_date)
    UNION ALL
    (SELECT do.drug_concept_id AS concept_id, do.drug_exposure_start_date AS start_date
    FROM dbo.drug_exposure do
    LEFT JOIN user_schema.dbo.iatrogenic_codes_with_desc i ON do.drug_concept_id = i.concept_id
    JOIN concept c ON do.drug_concept_id = c.concept_id
    LEFT JOIN visit_occurrence v ON do.visit_occurrence_id = v.visit_occurrence_id
    WHERE do.person_id = ? AND do.drug_concept_id != 0 AND i.concept_id IS NULL AND do.drug_exposure_start_date >= ? AND c.domain_id = 'Drug' 
        AND (v.visit_occurrence_id IS NULL OR DATEDIFF(DAY, do.drug_exposure_start_date, v.visit_end_date) <= 30)
    GROUP BY do.drug_concept_id, do.drug_exposure_start_date)
    UNION ALL
    (SELECT po.procedure_concept_id AS concept_id, po.procedure_date AS start_date
    FROM dbo.procedure_occurrence po
    LEFT JOIN user_schema.dbo.iatrogenic_codes_with_desc i ON po.procedure_concept_id = i.concept_id
    JOIN concept c ON po.procedure_concept_id = c.concept_id
    LEFT JOIN visit_occurrence v ON po.visit_occurrence_id = v.visit_occurrence_id
    WHERE po.person_id = ? AND po.procedure_concept_id != 0 AND i.concept_id IS NULL AND po.procedure_date >= ? AND 
        c.domain_id = 'Procedure' AND (v.visit_occurrence_id IS NULL OR po.procedure_date <= v.visit_end_date)
    GROUP BY po.procedure_concept_id, po.procedure_date)
    UNION ALL
    (SELECT m.measurement_concept_id AS concept_id, m.measurement_date AS start_date
    FROM dbo.measurement m
    LEFT JOIN user_schema.dbo.iatrogenic_codes_with_desc i ON m.measurement_concept_id = i.concept_id
    JOIN concept c ON m.measurement_concept_id = c.concept_id
    LEFT JOIN visit_occurrence v ON m.visit_occurrence_id = v.visit_occurrence_id
    WHERE m.person_id = ? AND m.measurement_concept_id != 0 AND i.concept_id IS NULL AND m.measurement_date >= ? AND c.domain_id = 'Measurement' 
        AND (v.visit_occurrence_id IS NULL OR DATEDIFF(DAY, m.measurement_date, v.visit_end_date) <= 30)
    GROUP BY m.measurement_concept_id, m.measurement_date)) x
GROUP BY x.start_date
ORDER BY x.start_date;
"""

def _serialize_sequence(res):
    return '\t'.join([x[0].strftime('%Y-%m-%d') + ':' + str(x[1]) for x in res])

with open(path.join(dir_data, 'patient_code_sequences.txt'), 'w') as fh:
    print('patient sequence progress: ')
    
    # Write out a header line with the format
    fh.write('Tab seperated cells. First cell is person_id. All following cells are YYYY-MM-DD:<concept IDs seperated by commas>')
    n_patients = 0
    for index, row in df_persons.iterrows():        
        # Get a sequence of the patient's codes and dates
        pid = row['person_id']
        dob = row['birth_date']
        cursor.execute(sql, pid, dob, pid, dob, pid, dob, pid, dob)
        res = cursor.fetchall()
        
        # Write the sequence if it's not empty
        if res:            
            sequence_str = _serialize_sequence(res)
            fh.write(str(pid) + '\t' + sequence_str + '\n')
            n_patients += 1
            
        # Display progress
        count = index + 1
        if count % 10000 == 0:
            percent = count / n_persons * 100
            elapsed_time = (time.time() - t1) / 60
            print(f'\t{percent:.02f}% - {elapsed_time:.01f} min')
        

elapsed_time = (time.time() - t1) / 60
print(elapsed_time)